In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility
import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv1D,MaxPooling1D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
matplotlib.use('agg')
import pylab as plt
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import Normalizer
from keras.utils import np_utils
import pickle
import h5py


# KDD - and - 
X_testN_ = pd.read_pickle("./dataset/kdd_test__2labels.pkl").as_matrix()
Y_testN_ = pd.read_pickle("./dataset/kdd_test__2labels_y.pkl").as_matrix()
X_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl").as_matrix()
Y_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels_y.pkl").as_matrix()
X_trainN_ = np.reshape(X_trainN_, (X_trainN_.shape[0],X_trainN_.shape[1],1))
X_testN_ = np.reshape(X_testN_, (X_testN_.shape[0],X_testN_.shape[1],1))

# KDD + and + 
X_testP_ = pd.read_pickle("./dataset/kdd_test_2labels.pkl").as_matrix()
Y_testP_ = pd.read_pickle("./dataset/kdd_test_2labels_y.pkl").as_matrix()
X_trainP_ = pd.read_pickle("./dataset/kdd_train_2labels.pkl").as_matrix()
Y_trainP_ = pd.read_pickle("./dataset/kdd_train_2labels_y.pkl").as_matrix()
X_trainP_ = np.reshape(X_trainP_, (X_trainP_.shape[0],X_trainP_.shape[1],1))
X_testP_ = np.reshape(X_testP_, (X_testP_.shape[0],X_testP_.shape[1],1))


# KDD - and + 
X_testP_ = pd.read_pickle("./dataset/kdd_test_2labels.pkl").as_matrix()
Y_testP_ = pd.read_pickle("./dataset/kdd_test_2labels_y.pkl").as_matrix()
X_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl").as_matrix()
Y_trainN_ = pd.read_pickle("./dataset/kdd_train__2labels_y.pkl").as_matrix()
X_trainN_ = np.reshape(X_trainN_, (X_trainN_.shape[0],X_trainN_.shape[1],1))
X_testP_ = np.reshape(X_testP_, (X_testP_.shape[0],X_testP_.shape[1],1))


# KDD + and - 
X_testN_ = pd.read_pickle("./dataset/kdd_test_2labels.pkl").as_matrix()
Y_testN_ = pd.read_pickle("./dataset/kdd_test_2labels_y.pkl").as_matrix()
X_trainP_ = pd.read_pickle("./dataset/kdd_train__2labels.pkl").as_matrix()
Y_trainP_ = pd.read_pickle("./dataset/kdd_train__2labels_y.pkl").as_matrix()
X_trainP_ = np.reshape(X_trainP_, (X_trainP_.shape[0],X_trainP_.shape[1],1))
X_testN_ = np.reshape(X_testN_, (X_testN_.shape[0],X_testN_.shape[1],1))


C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
cnn1d_1 = Sequential([Conv1D(64, 3, padding="same",input_shape=(124, 1)),
    Activation('relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])
print(cnn1d_1.summary())

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 124, 64)           256       
_________________________________________________________________
activation_1 (Activation)    (None, 124, 64)           0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 62, 64)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3968)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               508032    
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
___________________________________________________________

In [4]:
#cnn1d_1 = multi_gpu_model(cnn1d_1, gpus=8)
cnn1d_1.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
# train
start_time = time.time()
history = cnn1d_1.fit(X_trainP_, Y_trainP_, batch_size=64, validation_data=(X_testP_, Y_testP_) ,epochs=50)
end_time = time.time() 
print("Total time taken to train the Training model is", (end_time - start_time))
# serialize model to JSON
model_json = cnn1d_1.to_json()
with open("PP.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
cnn1d_1.save_weights("PP.h5")
print("Saved model to disk")
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Acc_ValaccuaracyPP.png')

Train on 25192 samples, validate on 22544 samples
Epoch 1/50
25192/25192 [==============================] - 7s 276us/step - loss: 0.0212 - acc: 0.9982 - val_loss: 0.4935 - val_acc: 0.9692
Epoch 2/50
25192/25192 [==============================] - 7s 269us/step - loss: 0.0371 - acc: 0.9973 - val_loss: 0.0334 - val_acc: 0.9979
Epoch 3/50
25192/25192 [==============================] - 7s 268us/step - loss: 0.0461 - acc: 0.9967 - val_loss: 0.1835 - val_acc: 0.9881
Epoch 4/50
25192/25192 [==============================] - 7s 280us/step - loss: 0.0247 - acc: 0.9982 - val_loss: 0.4906 - val_acc: 0.9693
Epoch 5/50
25192/25192 [==============================] - 7s 273us/step - loss: 0.0239 - acc: 0.9983 - val_loss: 0.4883 - val_acc: 0.9696
Epoch 6/50
25192/25192 [==============================] - 7s 273us/step - loss: 0.0181 - acc: 0.9987 - val_loss: 0.5058 - val_acc: 0.9685
Epoch 7/50
25192/25192 [==============================] - 7s 280us/step - loss: 0.0190 - acc: 0.9988 - val_loss: 0.5008 - 